## Using your sampler for a real problem where a grid would have been really painful.

### Intro: Gravitational microlensing

If you are a non-astronomer, you may or may not be familiar with **gravitational lensing**. Essentially, gravitational lensing occurs whenever a bright object appears behind a massive, dense object like a black hole or cluster of galaxies. This massive object is called the "lens", and gravity from the lens bends the light from the object in back. The result is that -- to observers on Earth -- the background object can appear distorted, and its light magnified.

**Example:** Below is a real image of gravitational lensing. Light from a blue spiral galaxy in the background is being magnified and warped by a red elliptical galaxy closer to us.

<table>
    <tr>
        <td><img src="figures/A_Horseshoe_Einstein_Ring_from_Hubble.JPG" width=30%></td>
    </tr>
</table>

For astronomers, it is easiest to detect gravitational lensing when the lens is an EXTREMELY large object -- typically a cluster of galaxies with trillions of times the mass of the Sun. However, we have now built survey telescopes to scan the sky for lensing from extremely small sources as well, including stars and planets that are small fractions of the Sun's mass. When the lens object is a single star or planet (rather than a whole galaxy), we call it a **microlensing event**.

To see this in action, [this microlensing demonstration from Ohio State University](http://www.astronomy.ohio-state.edu/~gaudi/Movies/point.gif) shows a background star (red) passing behind a lens star (yellow). The light from the background star is both warped and magnified by the lens star.

You can read more about microlensing at the links below if you like:
- https://en.wikipedia.org/wiki/Gravitational_microlensing
- http://www.astro.caltech.edu/~george/ay20/eaa-microlensing.pdf

### The task

We'll be analyzing data from the Optical Gravitational Lensing Experiment (OGLE), which monitors stars in our galaxy in the hopes of detecting gravitational microlensing events that occur when a compact mass (e.g. a fainter star) passes in front of the monitored star.

Data are available through the [OGLE Early Warning System](http://ogle.astrouw.edu.pl/ogle4/ews/ews.html). Scroll down a bit to the list of recent events and choose one to analyze. (Not the one shown below. Be original.) The event summary page will include a plot like this.

<table>
    <tr>
        <td><img src="http://ogle.astrouw.edu.pl/ogle4/ews/2019/data/2019/blg-0001/lcurve.gif" width=75%></td>
    </tr>
</table>

As long as a vaguely reasonable looking magenta line is shown, this should be a good data set to fit. Download the `phot.dat` for your chosen event (this file is linked at the bottom of the webpage, under the name "Photometry data file").

As described on the OGLE page, the columns of this text file are

> Hel.JD, I magnitude, magnitude error, seeing estimation (in pixels - 0.26"/pixel) and sky level

* Heliocentric Julian Date. This is time, measured in days, since a fixed reference. The "heliocentric" part means that it has been corrected to the reference frame of the Sun, i.e. the few minutes of light travel time more or less that would affect photon arrivals at different parts of the Earth's year have been subtracted off.

* Measurements of magnitude in the $I$ band (a near infrared band). Recall that astronomical magnitude, relative to a given reference source, is given by the relationship $m = m_\mathrm{ref} - 2.5\,\log_{10}\left(\frac{F}{F_\mathrm{ref}}\right)$, where $F$ is flux.

* Measurement uncertainty on the $I$ magnitude, defined in some unspecified way (digging through papers might elucidate this).

* The "seeing" and "sky level" quantities refer to the observing conditions, which we will not work with directly. These will have been accounted for (somehow) in deriving the best-fitting magnitude and its uncertainty.


As Bayesians we have 4 questions to answer. 
1. What's the model?
2. What's the Likelihood?
3. What's the Prior on the model?
4. How do you draw samples from the Posterior? 


I'll tell you the model (question 1), and you've already written down the answer to question 4 in homework 3.

You will be modeling the **flux of a star over time, $F(t)$**. In astronomy, this called a **lightcurve**.

Given a base flux $F_0$ for an object, a microlensing event can be modeled by the following lightcurve: 

## $$F(t) = F_0 \frac{u(t)^2 + 2}{u(t)\sqrt{u(t)^2+4}}$$

where

## $$u(t) = \sqrt{p^2 + \left( \frac{t-t_\mathrm{max}}{t_\mathrm{E}} \right)^2}$$

The function $u(t)$ has 3 parameters you'll need to fit:

* $t_\mathrm{max}$ is the time of maximum. You should be able to read this off from a plot of the lightcurve

* $t_\mathrm{E}$ is called the "Einstein crossing time". You can get an initial guess for this from the plot

* $p$ is the peak magnification. You can also get a guess of this and $F_0$ from the data.

For convenience, let's parameterize the normalization of the model lightcurve in magnitudes rather than flux, i.e. $I_0$ rather than $F_0$; that way, all of the "ref" quantities in the magnitude definition are absorbed into this new parameter and we won't have to worry about them explicitly. With that substitution, the model parameters are $I_0$, $p$, $t_\mathrm{max}$ and $t_\mathrm{E}$. You'll of course also need the transformation between flux and magnitude, above.

### I suggest making plots of your model with a variety of parameters to understand how the model behaves as a function of the parameters above. Since you didn't come up with the model (Bohdan Paczyinski did in 1986) this is a useful thing to do, if only to see how it behaves qualitatively.


Lacking any better information, we'll assume that the sampling distributions for the magnitude measurements are Gaussian and independent, with means given by the "magnitude" column and standard deviations given by the "magnitude error" column, and that the time stamps are exact.

Do an MCMC fit of this microlensing model to your lightcurve data. This fit should be doable with your Metropolis-Hastings implementation. To do so, you'll need to have your MCMC walker return a series of random samples drawn from the posterior distribution on your model (the prior * the likelihood). Then, you'll need to return the "best-fit" value: The sample from your MCMC walk for which the posterior probability was the highest. This sample will be your **best-fit model** for the microlensing event. (This is extremely useful data for astronomers. They would be able to take this model and determine the properties of the lens mass, such as its total mass, its radius, and its offset from the background source.)

---

### IMPORTANT NOTE: Log probabilities

This week, you'll need to change something about your Metropolis-Hastings function. Last week your function took in a posterior probability function `func`, and evaluated it to obtain the posterior probability `p`. This worked when your probabilities were on the order of 10^-10 at the smallest. It will NOT work this week: For some models, your likelihood function will be on the order of 10^-1000 or smaller! Python cannot represent numbers this small, and rounds them to 0.

**Instead of working with posterior probabilities, you'll need to make your Metropolis-Hastings function work with log probabilities.** To do so, you'll need to make two changes to your function.
1. Instead of taking in a function `func(x) -> posterior`, your Metropolis-Hastings implementation should take in a function `logfunc(x) -> log_posterior`. In other words, the function your Metropolis-Hastings implementation takes in should calculate the **log** of the posterior.
2. Instead of calculating the ratio $p(x') / p(x)$, your implementation should calculate $e\text{^}[ \text{log_p}(x') - \text{log_p}(x) ]$. (Mathematically, these expressions are equal, but Python will round $p(x)/p(x')$ to $0/0$ if these probabilities are very small.)

This should only require you to change a few lines in your Metropolis-Hastings function.

---

Your solution should include the following:

1. **Code:** Expressions (in readable code, at a minimum) of the prior distributions and likelihood encoded. (I.e. question 2 for a Bayesian)
2. **Writing:** A justification for your choice of priors (question 3 for a Bayesian). Be sure to explain why your choice of priors are "uninformative". 
3. **Plot:** Plots showing the traces of each parameter. On these plots, you should identify the "burn-in period".
    - The "burn-in period" is the time it takes for your MCMC model to reach a stable state. Generally, it takes on the order of ~100-200 or so samples for this to happen. You can identify this period visually by looking at a plot of your walker's location over time.
4. **Writing:** Some evaluation of how well the fit has converged (see below).
5. **Plot:** 1D histograms of the parameter samples and 2D contour plots of parameter pairs. This is generally called a "corner plot" (In addition to doing this by hand as we have in class and on the homework, there are also packages like `corner` and others to do this, feel free to use what you like.)
6. **Writing:** "Best fit" values and 68.3% confidence intervals from the 1D marginalized posteriors of each parameter. 
7. **Plot + Writing:** A plot of the best-fitting model lightcurve over the data, and some qualitative comments about how good a fit it appears to be. (Hint: Depending on your data set, you may need to zoom in quite a lot to get a good look)

In [3]:
# YOUR CODE HERE

# Optional (but very helpful to us) feedback form

https://forms.gle/N2s7qv3fEVY67Toe7